# 뉴럴 네트워크 학습 알고리즘 구현

## Import modules

- 보통은 Keras로 하지만, 지금은 학습목적으로 tensorflow방식 그대로 사용

In [1]:
import tensorflow as tf

## 하이퍼파라미터 정의

In [2]:
EPOCHS = 10

## 네트워크 구조 정의

In [3]:
class MyModel(tf.keras.Model):
    def __init__(self):
        super().__init__()  ## 상속자인 tf.keras.Model 클래스의 __init__ 해주기
        self.flatten = tf.keras.layers.Flatten(input_shape=(28,28))  ## 28 X 28행렬데이터를 벡터형태로 flatten
        self.dense1 = tf.keras.layers.Dense(32, activation='relu') ## 첫번째 layer에는 32개의 뉴런 사용
        self.dense2 = tf.keras.layers.Dense(64, activation='relu')
        self.dense3 = tf.keras.layers.Dense(128, activation='relu')
        self.dense4 = tf.keras.layers.Dense(256, activation='relu')
        self.dense5 = tf.keras.layers.Dense(10, activation='softmax')
        
    def call(self, x, trainig=None, mask=None):
        x = self.flatten(x)
        x = self.dense2(x)
        x = self.dense3(x)
        x = self.dense4(x)
        return self.dense5(x)

## 학습 함수 구현

- image : 입력값
- labels : 출력값

In [4]:
@tf.function
def train_step(model, image, labels, loss_object, optimizer, train_loss, train_accuracy):
    with tf.GradientTape() as tape: ## 이 안에서 하는 연산은 모두 미분을 계산해놓음  -> 나중에 편리하게 뽑아쓸 수 있음
        predictions = model(image) # batch_size=32. label종류=10가지 일때 --> (32 X 10) 의 형태로 나올 것.
        loss = loss_object(labels, predictions) 
    gradients = tape.gradient(loss, model.trainable_variables) # loss를 모든 trainable_variables로 미분
    optimizer.apply_gradients(zip(gradients, model.trainable_variables)) # zip()  ->  gradients와 trainable_variables를 같이 iteration 할 수 있는 iteration 객체
        
    train_loss(loss)
    train_accuracy(labels, predictions)

## 테스트 함수 구현

In [5]:
@tf.function
def test_step(model, image, labels, loss_object, test_loss, test_accuracy):
    predictions = model(image) # batch_size=32. label종류=10가지 일때 --> (32 X 10) 의 형태로 나올 것.
    loss = loss_object(labels, predictions) 
    
    test_loss(loss)
    test_accuracy(labels, predictions)

## 데이터 불러오기

In [6]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(1024).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)


## 모델 생성

In [7]:
model = MyModel()

## 손실함수 및 최적화 알고리즘 정의

In [8]:
print(y_train, y_train.shape) ## y의 범주들이 one-hot encoding되어있지 않고 각각이 범주이름을 나타낸다 --> SparseCategoricalCrossentropy 사용

loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

[5 0 4 ... 5 6 8] (60000,)


## 성능 지표 정의

In [9]:
train_loss = tf.keras.metrics.Mean(name="train_loss")
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name="train_accuracy")

test_loss = tf.keras.metrics.Mean(name="test_loss")
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name="test_accuracy")

## 학습 루프 구현

In [20]:
for epoch in range(EPOCHS):
    for image, labels in train_ds:
        train_step(model, image, labels, loss_object, optimizer, train_loss, train_accuracy)
        
    for image, labels in test_ds:
        test_step(model, image, labels, loss_object, test_loss, test_accuracy)
    
    print('Epoch {}: loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'.format(epoch+1, 
                                                                                       train_loss.result(), 
                                                                                       train_accuracy.result() * 100, 
                                                                                       test_loss.result(), 
                                                                                       test_accuracy.result() * 100))
    
    ## 누적되는것을 방지하기 위해 reset해줌
    train_loss.reset_states()
    train_accuracy.reset_states()

    test_loss.reset_states()
    test_accuracy.reset_states()

Epoch 1: loss: 0.024603623896837234, Accuracy: 99.20160675048828, Test Loss: 0.11980361491441727, Test Accuracy: 97.43000030517578
Epoch 2: loss: 0.019587818533182144, Accuracy: 99.3550033569336, Test Loss: 0.10614775121212006, Test Accuracy: 97.97999572753906
Epoch 3: loss: 0.021768029779195786, Accuracy: 99.29833221435547, Test Loss: 0.10762132704257965, Test Accuracy: 97.64999389648438
Epoch 4: loss: 0.01919660158455372, Accuracy: 99.37166595458984, Test Loss: 0.1312568634748459, Test Accuracy: 97.58999633789062
Epoch 5: loss: 0.01940007507801056, Accuracy: 99.39167022705078, Test Loss: 0.14052623510360718, Test Accuracy: 97.1500015258789
Epoch 6: loss: 0.017693521454930305, Accuracy: 99.41166687011719, Test Loss: 0.12548251450061798, Test Accuracy: 97.87999725341797
Epoch 7: loss: 0.016143256798386574, Accuracy: 99.50167083740234, Test Loss: 0.11711866408586502, Test Accuracy: 97.8499984741211
Epoch 8: loss: 0.016697455197572708, Accuracy: 99.45832824707031, Test Loss: 0.1374859064